In [1]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt


In [3]:
key_path = os.getenv('KEY_PATH')

# [START bigquery_client_json_credentials]

# TODO(developer): Set key_path to the path to the service account key
#                  file.
# key_path = "path/to/service_account.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
# [END bigquery_client_json_credentials]


In [4]:
# query = '''SELECT * FROM `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
# WHERE Timestamp BETWEEN DATETIME("2023-08-17")
# AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY) and Leg_0_StopsCount is not null'''

query = '''Select * from `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
WHERE Timestamp BETWEEN DATETIME("2023-08-17")AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY)
ORDER BY RAND()
LIMIT 200000
'''


In [5]:
query_job = client.query(query)

results = query_job.result()

the_data = results.to_dataframe()


In [6]:
the_data


,Guid,Timestamp,Market,Locale,Currency,OriginAirport,OriginCity,OriginCountry,DestinationAirport,DestinationCity,...,Leg_1_segment_3_OriginName,Leg_1_segment_3_DestinationIATA,Leg_1_segment_3_DestinationName,Leg_1_segment_3_DepartureTimestamp,Leg_1_segment_3_ArrivalTimestamp,Leg_1_segment_3_MarketingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_1_segment_3_FlightNumber,Leg_1_segment_3_DurationMin,Leg_1_segment_3_TravelDistanceKm
0,7c192b36-3dd1-11ee-bfb9-baa37deb1750,2023-08-18 14:14:03,UK,en-GB,GBP,LHR,LON,GB,HKG,HKG,...,,,,NaT,NaT,,,NaN,NaN,NaN
1,1018bbd4-3df1-11ee-8475-16b8a8f7f764,2023-08-18 18:00:06,UK,en-GB,GBP,ATH,ATH,GR,,,...,,,,NaT,NaT,,,NaN,NaN,NaN
2,8665e51a-3dc8-11ee-a5e4-5a091400f49d,2023-08-18 13:09:55,HK,zh-TW,HKD,HKG,HKG,CN,ICN,SEL,...,,,,NaT,NaT,,,NaN,NaN,NaN
3,94a19158-3df3-11ee-99cb-16b8a8f7f764,2023-08-18 18:18:07,UK,en-GB,MXN,MEX,MEX,MX,LHR,LON,...,,,,NaT,NaT,,,NaN,NaN,NaN
4,3a4a3652-3d8e-11ee-9906-b6d209ffa710,2023-08-18 06:12:36,IN,en-GB,INR,BBI,BBI,IN,BOM,BOM,...,,,,NaT,NaT,,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,f66fdc80-3d87-11ee-b898-de921c027a52,2023-08-18 05:27:45,US,en-US,INR,CGN,CGN,DE,ZRH,ZRH,...,,,,NaT,NaT,,,NaN,NaN,NaN
199996,9899333a-3ce3-11ee-8913-46c245b1fc4c,2023-08-17 09:51:11,DE,de-DE,EUR,AYT,AYT,TR,HAM,HAM,...,,,,NaT,NaT,,,NaN,NaN,NaN
199997,2b3728f0-3c94-11ee-9bfd-4a686d70cc18,2023-08-17 00:22:37,FR,fr-FR,EUR,AAE,AAE,DZ,ALG,ALG,...,,,,NaT,NaT,,,NaN,NaN,NaN
199998,24672cd0-3dd6-11ee-af0b-ceb199f889b2,2023-08-18 14:47:23,AU,en-GB,AUD,PER,PER,AU,BHX,BHX,...,,,,NaT,NaT,,,NaN,NaN,NaN


In [38]:
the_data['ItinerarySortType'].value_counts(normalize=True)


BEST                      0.535595
                          0.359240
CHEAPEST                  0.095055
FASTEST                   0.008210
DEPARTURE_OUTBOUND_ASC    0.001850
DEPARTURE_INBOUND_ASC     0.000050
Name: ItinerarySortType, dtype: float64

In [54]:
the_data['CabinClass'].value_counts(normalize=True)


ECONOMY           0.986870
BUSINESS          0.009385
PREMIUMECONOMY    0.003325
FIRST             0.000420
Name: CabinClass, dtype: float64

In [59]:
colums_subset = ['Market', 'OriginAirport', 'DestinationAirport', 'Locale', 'CabinClass', 'Kind',
          'TripLengthNights', 'TravelType', 'TravelDistanceKm', 'TravelHorizonDays', 'RedirectType', 'ItineraryPosition', 'ItinerarySortType',
          'PriceCustomCurrencyPerPax',
          'Leg_0_StopsCount', 'Leg_1_StopsCount',
          'Leg_0_segment_0_OperatingCarrierIATA', 'Leg_0_segment_1_OperatingCarrierIATA', 'Leg_0_segment_2_OperatingCarrierIATA', 'Leg_0_segment_3_OperatingCarrierIATA',
          'Leg_1_segment_0_OperatingCarrierIATA', 'Leg_1_segment_1_OperatingCarrierIATA', 'Leg_1_segment_2_OperatingCarrierIATA', 'Leg_1_segment_3_OperatingCarrierIATA',
          'Leg_0_DurationMin', 'Leg_1_DurationMin'
]

all_columns = list(the_data.columns)


In [64]:
remaining_columns = list(set(all_columns) - set(colums_subset))
remaining_columns


['Leg_0_segment_3_DepartureTimestamp',
 'Leg_0_segment_2_ArrivalTimestamp',
 'Timestamp',
 'Leg_0_segment_0_OriginName',
 'Leg_1_segment_2_DestinationIATA',
 'UserCountryName',
 'SelfTransfer',
 'Leg_1_segment_2_TravelDistanceKm',
 'ProviderRank',
 'Leg_1_segment_0_TravelDistanceKm',
 'Leg_0_segment_3_DestinationIATA',
 'ProviderType',
 'ProviderPreference',
 'PassengerChild',
 'Leg_1_segment_0_OriginName',
 'ItineraryLowestPriceRatio',
 'Leg_1_segment_1_OriginName',
 'Leg_1_segment_1_ArrivalTimestamp',
 'Leg_1_segment_3_DestinationName',
 'Leg_0_segment_2_OriginIATA',
 'Guid',
 'Leg_1_segment_2_DurationMin',
 'Leg_0_segment_1_DestinationIATA',
 'Leg_0_segment_0_DepartureTimestamp',
 'Leg_0_segment_2_FlightNumber',
 'SessionId',
 'PassengerInfant',
 'Leg_1_segment_3_DepartureTimestamp',
 'PriceUserCurrency',
 'Leg_1_segment_1_MarketingCarrierIATA',
 'BrowserVersion',
 'Leg_0_segment_3_OriginIATA',
 'Leg_0_segment_3_DestinationName',
 'Leg_1_segment_2_OriginName',
 'Leg_1_segment_2_Arri

In [73]:
data_subset = the_data[colums_subset].copy()
data_subset


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_0_OperatingCarrierIATA,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin
0,UK,MAN,MXP,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,1168.0,26,...,FR,,,,,,,,130,NaN
1,SA,CAI,RUH,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,1611.0,34,...,SV,,,,,,,,175,NaN
2,AT,VIE,BCN,de-DE,ECONOMY,RETURN,2.0,INTERNATIONAL,1369.0,119,...,LW,,,,LW,,,,140,150.0
3,IN,BLR,VTZ,en-GB,ECONOMY,ONE_WAY,NaN,DOMESTIC,776.0,18,...,I5,,,,,,,,80,NaN
4,ES,MAD,BVA,es-ES,ECONOMY,RETURN,2.0,INTERNATIONAL,1092.0,30,...,FR,,,,FR,,,,125,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,PT,OPO,FAO,pt-PT,ECONOMY,RETURN,4.0,DOMESTIC,474.0,10,...,FR,,,,FR,,,,75,70.0
199996,JP,HND,ITM,ja-JP,ECONOMY,ONE_WAY,NaN,DOMESTIC,402.0,0,...,JL,,,,,,,,65,NaN
199997,IN,BLR,IXR,en-GB,ECONOMY,ONE_WAY,NaN,DOMESTIC,1382.0,14,...,6E,,,,,,,,150,NaN
199998,KR,ICN,KIX,ko-KR,ECONOMY,RETURN,4.0,INTERNATIONAL,861.0,53,...,OZ,,,,OZ,,,,105,110.0


In [74]:
data_subset['OD'] = data_subset['OriginAirport'] + data_subset['DestinationAirport']


In [76]:
data_subset[data_subset['CabinClass'] == 'ECONOMY']


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin,OD
0,UK,MAN,MXP,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,1168.0,26,...,,,,,,,,130,NaN,MANMXP
1,SA,CAI,RUH,en-GB,ECONOMY,ONE_WAY,NaN,INTERNATIONAL,1611.0,34,...,,,,,,,,175,NaN,CAIRUH
2,AT,VIE,BCN,de-DE,ECONOMY,RETURN,2.0,INTERNATIONAL,1369.0,119,...,,,,LW,,,,140,150.0,VIEBCN
3,IN,BLR,VTZ,en-GB,ECONOMY,ONE_WAY,NaN,DOMESTIC,776.0,18,...,,,,,,,,80,NaN,BLRVTZ
4,ES,MAD,BVA,es-ES,ECONOMY,RETURN,2.0,INTERNATIONAL,1092.0,30,...,,,,FR,,,,125,130.0,MADBVA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,PT,OPO,FAO,pt-PT,ECONOMY,RETURN,4.0,DOMESTIC,474.0,10,...,,,,FR,,,,75,70.0,OPOFAO
199996,JP,HND,ITM,ja-JP,ECONOMY,ONE_WAY,NaN,DOMESTIC,402.0,0,...,,,,,,,,65,NaN,HNDITM
199997,IN,BLR,IXR,en-GB,ECONOMY,ONE_WAY,NaN,DOMESTIC,1382.0,14,...,,,,,,,,150,NaN,BLRIXR
199998,KR,ICN,KIX,ko-KR,ECONOMY,RETURN,4.0,INTERNATIONAL,861.0,53,...,,,,OZ,,,,105,110.0,ICNKIX


In [77]:
data_subset['Leg_0_segment_0_OperatingCarrierIATA'].value_counts(normalize=True)


FR    0.046690
      0.043260
6E    0.036975
U2    0.035780
UK    0.023045
        ...   
TJ    0.000005
KY    0.000005
6O    0.000005
S8    0.000005
RC    0.000005
Name: Leg_0_segment_0_OperatingCarrierIATA, Length: 550, dtype: float64